# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,18.39,55,0,3.15,AU,1726552203
1,1,el granada,37.5027,-122.4694,14.09,87,100,1.54,US,1726552204
2,2,grytviken,-54.2811,-36.5092,-1.42,77,41,5.30,GS,1726552206
3,3,san patricio,28.0170,-97.5169,23.97,78,0,3.41,US,1726552207
4,4,waitangi,-43.9535,-176.5597,9.34,96,100,1.34,NZ,1726552208


In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM',  
    size='Humidity', 
    scale=0.5, 
    color='City',  
    frame_width=700, 
    frame_height=500
)



# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Define ideal weather conditions
ideal_temp_min = 21
ideal_temp_max = 26

# Narrow down cities that fit criteria
filtered_cities = city_data_df.loc[
    (city_data_df['Max Temp'] >= ideal_temp_min) & 
    (city_data_df['Max Temp'] <= ideal_temp_max) & 
    (city_data_df["Cloudiness"] == 0) &
    (city_data_df["Wind Speed"] < 4.5)
]    


# Drop any rows with null values
filtered_cities = filtered_cities.dropna()
filtered_cities





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,san patricio,28.0170,-97.5169,23.97,78,0,3.41,US,1726552207
8,8,dalby,-27.1833,151.2667,23.52,18,0,3.21,AU,1726552212
39,39,stephenville,32.2207,-98.2023,24.07,75,0,3.09,US,1726552247
112,112,al bawiti,28.3492,28.8659,25.80,52,0,2.76,EG,1726552334
211,211,seminole,28.7086,-81.2081,24.59,88,0,1.54,US,1726552451
230,230,kas,36.2018,29.6377,25.29,56,0,1.63,TR,1726552474
334,334,kulgam,33.6500,75.0167,24.98,38,0,2.03,IN,1726552597
344,344,dalandzadgad,43.5708,104.4250,22.56,23,0,2.03,MN,1726552608
356,356,nova vicosa,-17.8919,-39.3719,22.80,85,0,3.80,BR,1726552622
429,429,upper grand lagoon,30.1633,-85.7408,24.45,91,0,2.06,US,1726552704


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = " "
# Display sample data
# YOUR CODE HERE
hotel_df.head(15)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,margaret river,AU,-33.9500,115.0667,55,
1,el granada,US,37.5027,-122.4694,87,
2,grytviken,GS,-54.2811,-36.5092,77,
3,san patricio,US,28.0170,-97.5169,78,
4,waitangi,NZ,-43.9535,-176.5597,96,
5,whitehorse,CA,60.7161,-135.0538,84,
6,albany,US,42.6001,-73.9662,72,
7,fale old settlement,TK,-9.3852,-171.2468,76,
8,dalby,AU,-27.1833,151.2667,18,
9,petrozavodsk,RU,61.7849,34.3469,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10,
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    print(f"\nProcessing: {city}")
   
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionary
    name_address_response = requests.get(base_url,params=params)
    # Convert the API response to JSON format
    name_address = name_address_response.json()
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except(KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df


Starting hotel search

Processing: margaret river
margaret river - nearest hotel: Margaret River Hotel

Processing: el granada
el granada - nearest hotel: Beach House

Processing: grytviken
grytviken - nearest hotel: No hotel found

Processing: san patricio
san patricio - nearest hotel: No hotel found

Processing: waitangi
waitangi - nearest hotel: Hotel Chathams

Processing: whitehorse
whitehorse - nearest hotel: Town & Mountain Hotel

Processing: albany
albany - nearest hotel: No hotel found

Processing: fale old settlement
fale old settlement - nearest hotel: No hotel found

Processing: dalby
dalby - nearest hotel: No hotel found

Processing: petrozavodsk
petrozavodsk - nearest hotel: Piter Inn

Processing: fada
fada - nearest hotel: No hotel found

Processing: hawaiian paradise park
hawaiian paradise park - nearest hotel: No hotel found

Processing: bang saphan
bang saphan - nearest hotel: Hotel

Processing: antofagasta
antofagasta - nearest hotel: Hotel Colón

Processing: nili
nil

fruita - nearest hotel: Balanced Rock Inn

Processing: magnitka
magnitka - nearest hotel: No hotel found

Processing: adamstown
adamstown - nearest hotel: No hotel found

Processing: saipan
saipan - nearest hotel: Chalan Kanoa Beach Hotel

Processing: santa elena de uairen
santa elena de uairen - nearest hotel: Posada Villa Fairmont

Processing: carnarvon
carnarvon - nearest hotel: No hotel found

Processing: petropavlovsk-kamchatsky
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор

Processing: avarua
avarua - nearest hotel: Paradise Inn

Processing: kidatu
kidatu - nearest hotel: KWA KANYA

Processing: grand baie
grand baie - nearest hotel: Safari

Processing: nemuro
nemuro - nearest hotel: お宿エクハシ

Processing: east london
east london - nearest hotel: No hotel found

Processing: mar del plata
mar del plata - nearest hotel: Hotel 3 de Abril

Processing: tazovsky
tazovsky - nearest hotel: Тазовчанка

Processing: montabaur
montabaur - nearest hotel: Hotel Peterstor

Processing

oskarshamn - nearest hotel: Clarion Collection Hotel Post

Processing: arkalyk
arkalyk - nearest hotel: Гостиница "Арсенал"

Processing: revda
revda - nearest hotel: Металлург

Processing: kilindoni
kilindoni - nearest hotel: Ibizza Inn

Processing: lamu
lamu - nearest hotel: Hotel Archipelago villa bed and breakfast

Processing: funafuti
funafuti - nearest hotel: Vaiaku Lagi Hotel

Processing: pechora
pechora - nearest hotel: Центральная

Processing: mitu
mitu - nearest hotel: Hotel Brasilia Vaupés

Processing: manohar thana
manohar thana - nearest hotel: No hotel found

Processing: atherton
atherton - nearest hotel: Britannia Hotel

Processing: kot diji
kot diji - nearest hotel: No hotel found

Processing: korla
korla - nearest hotel: Silver Star Hotel

Processing: chita
chita - nearest hotel: No hotel found

Processing: olds
olds - nearest hotel: Best Western

Processing: qarqaraly
qarqaraly - nearest hotel: Каркаралинск

Processing: weligama
weligama - nearest hotel: Basecamp

Proc

benton harbor - nearest hotel: Silver Beach Hotel

Processing: fort mcmurray
fort mcmurray - nearest hotel: Nomad Hotel and Suites

Processing: genhe
genhe - nearest hotel: No hotel found

Processing: horta
horta - nearest hotel: Azoris Faial Garden - Resort Hotel

Processing: ribeira brava
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel

Processing: kerikeri
kerikeri - nearest hotel: Avalon Resort

Processing: belokurikha
belokurikha - nearest hotel: Home

Processing: giridih
giridih - nearest hotel: No hotel found

Processing: maragogi
maragogi - nearest hotel: Pousada Mariluz

Processing: beacon square
beacon square - nearest hotel: Fairfield Inn & Suites Holiday Tarpon Springs

Processing: box elder
box elder - nearest hotel: No hotel found

Processing: dakhla
dakhla - nearest hotel: Hotel Ayouz

Processing: antsiranana
antsiranana - nearest hotel: Central

Processing: krakeel
krakeel - nearest hotel: Hotel Hoogeveen

Processing: katsuura
katsuura - nearest hotel: 潮騒の宿

P

kill devil hills - nearest hotel: Mariner Days Inn & Suites

Processing: metlili chaamba
metlili chaamba - nearest hotel: No hotel found

Processing: poronaysk
poronaysk - nearest hotel: No hotel found

Processing: nieuw amsterdam
nieuw amsterdam - nearest hotel: Leonsberg Lodge

Processing: kuanda
kuanda - nearest hotel: No hotel found

Processing: halden
halden - nearest hotel: Grand Hotell

Processing: ellore
ellore - nearest hotel: Grand Aryas

Processing: coos bay
coos bay - nearest hotel: No hotel found

Processing: kota kinabalu
kota kinabalu - nearest hotel: Cititel Express

Processing: frohburg
frohburg - nearest hotel: Hotel Zur Post

Processing: brigantine
brigantine - nearest hotel: La Sammana

Processing: turpan
turpan - nearest hotel: Oriental

Processing: cabedelo
cabedelo - nearest hotel: No hotel found

Processing: krasnokamensk
krasnokamensk - nearest hotel: Аргунь

Processing: selfoss
selfoss - nearest hotel: Miðgarður

Processing: adi keyh
adi keyh - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,margaret river,AU,-33.9500,115.0667,55,Margaret River Hotel
1,el granada,US,37.5027,-122.4694,87,Beach House
2,grytviken,GS,-54.2811,-36.5092,77,No hotel found
3,san patricio,US,28.0170,-97.5169,78,No hotel found
4,waitangi,NZ,-43.9535,-176.5597,96,Hotel Chathams
...,...,...,...,...,...,...
583,berezovyy,RU,51.6667,135.7000,39,No hotel found
584,portland,US,45.5234,-122.6762,77,the Hoxton
585,maitum,PH,6.0392,124.4986,62,No hotel found
586,staroshcherbinovskaya,RU,46.6311,38.6742,40,"Гостиница ""Белая Акация"""


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=700, 
    size=10, 
    color='City', 
    hover_cols=['Hotel Name','Humidity', 'City','Country']
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Humidity,Country)